# Exercise for Unit 4

**Name:** MYRRHEA BELLE B. JUNSAY

**Date:** OCTOBER 17, 2025  

**Year and Section:** BSCS 3A AI

INSTALL THE FOLLOWING PYTHON PACKAGES FIRST BEFORE RUNNING THE PROGRAM

1) Numpy
2) NNFS - for the Spiral dataset
3) scikit-learn - for the iris dataset

In [9]:
%pip install numpy nnfs scikit-learn

In [10]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

Create classes for modularity

In [11]:
# Hidden Layers
# Dense
class Layer_Dense:
    # Layer initialization
    # randomly initialize weights and set biases to zero
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
        self.weight_momentums = np.zeros_like(self.weights)
        self.bias_momentums = np.zeros_like(self.biases)
        self.weight_cache = np.zeros_like(self.weights)
        self.bias_cache = np.zeros_like(self.biases)

    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)

In [12]:
# Activation Functions
# Included here are the functions for both the forward and backward pass

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [13]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and ground truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = len(y_pred)

        # Clip data to prevent division by 0
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values
        # Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate the gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples

This cell has been modified to include all three optimizers for Task 2.

In [14]:
# SGD optimizer with learning rate decay and momentum
class Optimizer_SGD:
    # Initialize optimizer with learning rate, decay, and momentum
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0):
        self.initial_learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    # Call this before any parameter updates to apply learning rate decay
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.initial_learning_rate * (1. / (1. + self.decay * self.iterations))

    # Update the parameters
    def update_params(self, layer):
        # If we use momentum
        if self.momentum:
            # Create weight updates with momentum
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            
            # Create bias updates with momentum
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
        
        # Vanilla SGD updates (if no momentum)
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases
            
        # Update weights and biases
        layer.weights += weight_updates
        layer.biases += bias_updates

    # Call this after any parameter updates to increment our iteration counter
    def post_update_params(self):
        self.iterations += 1

# Adagrad optimizer (Adaptive Gradient)
class Optimizer_Adagrad:
    # Initialize optimizer
    def __init__(self, learning_rate=1.0, decay=0.0, epsilon=1e-7):
        self.initial_learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    # Call this before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.initial_learning_rate * (1. / (1. + self.decay * self.iterations))

    # Update the parameters
    def update_params(self, layer):
        # Update cache with squared current gradients
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2
        
        # Update weights and biases using the Adagrad formula
        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    # Call this after any parameter updates
    def post_update_params(self):
        self.iterations += 1

We can use a sample dataset from the Spiral module.

In [15]:
# Spiral Data
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples = 100, classes = 3)

This final code block has been modified to perform the full training loop for Tasks 1, 3, and 4.

In [20]:
# Neural Network Initialization
dense1 = Layer_Dense(2, 64) 
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 3) 
activation2 = Activation_Softmax()
loss_function = Loss_CategoricalCrossEntropy()

# Optimizer a: SGD with Learning Rate Decay
#optimizer = Optimizer_SGD(learning_rate=1.0, decay=1e-3)

# Optimizer b: SGD with Momentum
#optimizer = Optimizer_SGD(learning_rate=0.1, decay=1e-3, momentum=0.9)

# Optimizer c: Adagrad (Adaptive Gradient)
optimizer = Optimizer_Adagrad(learning_rate=1.0, decay=1e-4)


for epoch in range(1001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    loss = loss_function.calculate(activation2.output, y)

    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions == y)
    
    if epoch % 100 == 0:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')

    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.pre_update_params() 
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params() 

epoch: 0, acc: 0.333, loss: 1.099, lr: 1.0
epoch: 100, acc: 0.517, loss: 0.935, lr: 0.9901970492127933
epoch: 100, acc: 0.517, loss: 0.935, lr: 0.9901970492127933
epoch: 200, acc: 0.570, loss: 0.876, lr: 0.9804882831650161
epoch: 200, acc: 0.570, loss: 0.876, lr: 0.9804882831650161
epoch: 300, acc: 0.590, loss: 0.844, lr: 0.9709680551509855
epoch: 300, acc: 0.590, loss: 0.844, lr: 0.9709680551509855
epoch: 400, acc: 0.573, loss: 0.812, lr: 0.9616309260505818
epoch: 400, acc: 0.573, loss: 0.812, lr: 0.9616309260505818
epoch: 500, acc: 0.640, loss: 0.778, lr: 0.9524716639679969
epoch: 500, acc: 0.640, loss: 0.778, lr: 0.9524716639679969
epoch: 600, acc: 0.720, loss: 0.703, lr: 0.9434852344560807
epoch: 600, acc: 0.720, loss: 0.703, lr: 0.9434852344560807
epoch: 700, acc: 0.717, loss: 0.665, lr: 0.9346667912889054
epoch: 700, acc: 0.717, loss: 0.665, lr: 0.9346667912889054
epoch: 800, acc: 0.703, loss: 0.658, lr: 0.9260116677470135
epoch: 800, acc: 0.703, loss: 0.658, lr: 0.92601166774701